In [2]:
import csv
import pandas as pd
sample_df = pd.read_csv("39A_rt_trips_2017_I_DB.csv")
sample_df.head()

,Unnamed: 0,datasource,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,basin,tenderlot,suppressed,justificationid,lastupdate,note
0,0,DB,24-FEB-17 00:00:00,4388883,39A,39A_40,1,34126,29400,34777.0,29619.0,BasDef,NaN,NaN,NaN,05-MAR-17 02:56:33,",2417274,"
1,1,DB,24-FEB-17 00:00:00,4388888,39A,39A_43,2,65671,60600,66094.0,60593.0,BasDef,NaN,NaN,NaN,05-MAR-17 02:56:33,",2417282,"
2,2,DB,24-FEB-17 00:00:00,4391989,39A,39A_40,1,51616,46800,NaN,46787.0,BasDef,NaN,NaN,NaN,05-MAR-17 02:56:33,",2426175,"
3,3,DB,24-FEB-17 00:00:00,4387515,39A,39A_43,2,32015,26400,30946.0,NaN,BasDef,NaN,NaN,NaN,05-MAR-17 02:56:33,",2416628,"
4,4,DB,10-MAR-17 00:00:00,4387515,39A,39A_43,2,32015,26400,32363.0,NaN,BasDef,NaN,NaN,NaN,16-MAR-17 11:26:26,",2416628,"


In [3]:
distance_df = pd.read_csv("39A_distance_b.csv")
distance_df.head()

,progrnumber,stoppointid,distance
0,1,7158,0
1,2,7048,400.0
2,3,7159,600.0
3,4,7388,200.0
4,5,7017,800.0


In [4]:
sample_df.drop_duplicates(subset=['tripid'],keep='first',inplace=True)

In [5]:
# This is only for 39A, _a is direction 1, _b is direction 2
tripids_a = []
tripids_b = []
for i in sample_df.index:
    if sample_df['direction'].loc[i] == 1:
        tripids_a.append(sample_df['tripid'].loc[i])
    elif sample_df['direction'].loc[i] == 2:
        tripids_b.append(sample_df['tripid'].loc[i])

In [6]:
print(tripids_b[:10])

[4388888, 4387515, 4384445, 4386521, 4387394, 4387396, 4387743, 4593583, 4583450, 4387745]


In [7]:
#trip_df = pd.read_csv("rt_leavetimes_2017_I_DB_4388883.csv")
#sample_trip_df = pd.read_csv("rt_leavetimes_2017_I_DB_4388883.csv")
trip_df = pd.read_csv("rt_leavetimes_2017_I_DB_39A_b.csv")
#sample_trip_df = pd.read_csv("rt_leavetimes_2017_I_DB_39A.csv")

In [8]:
trip_df.count()

Unnamed: 0         8429
datasource         8429
dayofservice       8429
tripid             8429
progrnumber        8429
stoppointid        8429
plannedtime_arr    8429
plannedtime_dep    8429
actualtime_arr     8429
actualtime_dep     8429
vehicleid          8429
passengers            0
passengersin          0
passengersout         0
distance              0
suppressed          264
justificationid     264
lastupdate         8429
note                  0
dtype: int64

In [9]:
trip_df.head()

,Unnamed: 0,datasource,dayofservice,tripid,progrnumber,stoppointid,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,vehicleid,passengers,passengersin,passengersout,distance,suppressed,justificationid,lastupdate,note
0,0,DB,20-FEB-17 00:00:00,4388888,1,7158,60600,60600,60585,60585,1000612,NaN,NaN,NaN,NaN,NaN,NaN,05-MAR-17 02:13:13,NaN
1,1,DB,20-FEB-17 00:00:00,4388888,34,1700,62592,62592,63201,63201,1000612,NaN,NaN,NaN,NaN,NaN,NaN,05-MAR-17 02:13:13,NaN
2,2,DB,20-FEB-17 00:00:00,4388888,51,335,63880,63880,64693,64782,1000612,NaN,NaN,NaN,NaN,NaN,NaN,05-MAR-17 02:13:13,NaN
3,3,DB,20-FEB-17 00:00:00,4388888,68,761,65359,65359,65929,65929,1000612,NaN,NaN,NaN,NaN,NaN,NaN,05-MAR-17 02:13:13,NaN
4,4,DB,20-FEB-17 00:00:00,4388888,2,7048,60642,60642,60619,60619,1000612,NaN,NaN,NaN,NaN,NaN,NaN,05-MAR-17 02:13:13,NaN


In [12]:
# we also need to add 'plannedPreTripDuration' and 'plannedTripDuration' into input features.
d = {'tripid': None,
     'dayInterval': [],
     'start_progrnumber': None,
     'end_progrnumber': None,
     'stoppointid': None,
     'nextstoppointid': None,
     'plannedPreTripDuration':[],
     'preTripDuration':[],
     'toNextDistance':[],
     'plannedTripDuration':[],
     'tripDuration': []}
new_df = pd.DataFrame(data=d)

In [13]:
for tripid in tripids_b[:10]:
    print(tripid)
    count = 0
    sample_trip_df = trip_df[(trip_df['tripid']==tripid)].copy()
    # Get list of 'dayofservice'
    sample_trip_df.drop_duplicates(subset=['dayofservice'], keep='first',inplace=True)
    days = []
    for i in sample_trip_df.index:
        days.append(sample_trip_df['dayofservice'].loc[i])
        
    for day in days:
        print(day)
        oneRound_trip_df = trip_df[(trip_df['tripid'] == tripid)&(trip_df['dayofservice'] == day)].copy()
        oneRound_trip_df.reset_index(inplace=True)
        for i in range(1,73):
       
            try:
                tripid = int(oneRound_trip_df.loc[0]['tripid'])
                t00 = oneRound_trip_df[(oneRound_trip_df['progrnumber'] == 1)].iloc[0]['actualtime_dep']
                t01 = oneRound_trip_df[(oneRound_trip_df['progrnumber'] == i)].iloc[0]['actualtime_arr']
                pt00 = oneRound_trip_df[(oneRound_trip_df['progrnumber'] == 1)].iloc[0]['plannedtime_dep']
                pt01 = oneRound_trip_df[(oneRound_trip_df['progrnumber'] == i)].iloc[0]['plannedtime_arr']
                t1 = oneRound_trip_df[(oneRound_trip_df['progrnumber'] == i)].iloc[0]['actualtime_dep']
                t2 = oneRound_trip_df[(oneRound_trip_df['progrnumber'] == (i+1))].iloc[0]['actualtime_arr']
                pt1 = oneRound_trip_df[(oneRound_trip_df['progrnumber'] == i)].iloc[0]['plannedtime_dep']
                pt2 = oneRound_trip_df[(oneRound_trip_df['progrnumber'] == (i+1))].iloc[0]['plannedtime_arr']
                stoppointid = oneRound_trip_df[(oneRound_trip_df['progrnumber'] == i)].iloc[0]['stoppointid']
                nextstoppointid = oneRound_trip_df[(oneRound_trip_df['progrnumber'] == (i+1))].iloc[0]['stoppointid']
                preTripDuration = t01 - t00
                plannedPreTripDuration = pt01 - pt00
                tripDuration = t2-t1
                plannedTripDuration = pt2 - pt1
                dayInterval = t1//3600
                toNextDistance = distance_df[(distance_df['progrnumber']==(i+1))].iloc[0]['distance']
                new_df = new_df.append({'tripid': tripid,
                                        'dayInterval': dayInterval,
                                        'start_progrnumber': i,
                                        'end_progrnumber': i+1,
                                        'stoppointid': stoppointid,
                                        'nextstoppointid': nextstoppointid,
                                        'preTripDuration': preTripDuration,
                                        'plannedTripDuration': plannedTripDuration,
                                        'toNextDistance': toNextDistance,
                                        'tripDuration': tripDuration,
                                        'plannedPreTripDuration': plannedPreTripDuration}, ignore_index=True)
                count += 1
            except IndexError:
                print("Got IndexError on: {}, start: {}, end: {}".format(day, i, i+1))
print("count: {}".format(count))

4388888
20-FEB-17 00:00:00
Got IndexError on: 20-FEB-17 00:00:00, start: 71, end: 72
Got IndexError on: 20-FEB-17 00:00:00, start: 72, end: 73
28-FEB-17 00:00:00
Got IndexError on: 28-FEB-17 00:00:00, start: 13, end: 14
Got IndexError on: 28-FEB-17 00:00:00, start: 14, end: 15
Got IndexError on: 28-FEB-17 00:00:00, start: 15, end: 16
Got IndexError on: 28-FEB-17 00:00:00, start: 16, end: 17
Got IndexError on: 28-FEB-17 00:00:00, start: 17, end: 18
Got IndexError on: 28-FEB-17 00:00:00, start: 18, end: 19
Got IndexError on: 28-FEB-17 00:00:00, start: 19, end: 20
Got IndexError on: 28-FEB-17 00:00:00, start: 20, end: 21
Got IndexError on: 28-FEB-17 00:00:00, start: 26, end: 27
Got IndexError on: 28-FEB-17 00:00:00, start: 27, end: 28
Got IndexError on: 28-FEB-17 00:00:00, start: 28, end: 29
Got IndexError on: 28-FEB-17 00:00:00, start: 29, end: 30
Got IndexError on: 28-FEB-17 00:00:00, start: 30, end: 31
Got IndexError on: 28-FEB-17 00:00:00, start: 31, end: 32
Got IndexError on: 28-FEB-

Got IndexError on: 22-FEB-17 00:00:00, start: 71, end: 72
Got IndexError on: 22-FEB-17 00:00:00, start: 72, end: 73
01-MAR-17 00:00:00
Got IndexError on: 01-MAR-17 00:00:00, start: 71, end: 72
Got IndexError on: 01-MAR-17 00:00:00, start: 72, end: 73
23-FEB-17 00:00:00
Got IndexError on: 23-FEB-17 00:00:00, start: 71, end: 72
Got IndexError on: 23-FEB-17 00:00:00, start: 72, end: 73
02-MAR-17 00:00:00
Got IndexError on: 02-MAR-17 00:00:00, start: 71, end: 72
Got IndexError on: 02-MAR-17 00:00:00, start: 72, end: 73
21-FEB-17 00:00:00
Got IndexError on: 21-FEB-17 00:00:00, start: 71, end: 72
Got IndexError on: 21-FEB-17 00:00:00, start: 72, end: 73
4387515
20-FEB-17 00:00:00
Got IndexError on: 20-FEB-17 00:00:00, start: 1, end: 2
Got IndexError on: 20-FEB-17 00:00:00, start: 2, end: 3
Got IndexError on: 20-FEB-17 00:00:00, start: 3, end: 4
Got IndexError on: 20-FEB-17 00:00:00, start: 4, end: 5
Got IndexError on: 20-FEB-17 00:00:00, start: 5, end: 6
Got IndexError on: 20-FEB-17 00:00:00

Got IndexError on: 28-FEB-17 00:00:00, start: 60, end: 61
Got IndexError on: 28-FEB-17 00:00:00, start: 61, end: 62
Got IndexError on: 28-FEB-17 00:00:00, start: 62, end: 63
Got IndexError on: 28-FEB-17 00:00:00, start: 63, end: 64
Got IndexError on: 28-FEB-17 00:00:00, start: 64, end: 65
Got IndexError on: 28-FEB-17 00:00:00, start: 65, end: 66
Got IndexError on: 28-FEB-17 00:00:00, start: 71, end: 72
Got IndexError on: 28-FEB-17 00:00:00, start: 72, end: 73
09-MAR-17 00:00:00
Got IndexError on: 09-MAR-17 00:00:00, start: 1, end: 2
Got IndexError on: 09-MAR-17 00:00:00, start: 2, end: 3
Got IndexError on: 09-MAR-17 00:00:00, start: 3, end: 4
Got IndexError on: 09-MAR-17 00:00:00, start: 4, end: 5
Got IndexError on: 09-MAR-17 00:00:00, start: 5, end: 6
Got IndexError on: 09-MAR-17 00:00:00, start: 6, end: 7
Got IndexError on: 09-MAR-17 00:00:00, start: 7, end: 8
Got IndexError on: 09-MAR-17 00:00:00, start: 8, end: 9
Got IndexError on: 09-MAR-17 00:00:00, start: 9, end: 10
Got IndexErr

Got IndexError on: 24-FEB-17 00:00:00, start: 67, end: 68
Got IndexError on: 24-FEB-17 00:00:00, start: 68, end: 69
Got IndexError on: 24-FEB-17 00:00:00, start: 69, end: 70
Got IndexError on: 24-FEB-17 00:00:00, start: 70, end: 71
Got IndexError on: 24-FEB-17 00:00:00, start: 71, end: 72
Got IndexError on: 24-FEB-17 00:00:00, start: 72, end: 73
03-MAR-17 00:00:00
Got IndexError on: 03-MAR-17 00:00:00, start: 1, end: 2
Got IndexError on: 03-MAR-17 00:00:00, start: 2, end: 3
Got IndexError on: 03-MAR-17 00:00:00, start: 3, end: 4
Got IndexError on: 03-MAR-17 00:00:00, start: 4, end: 5
Got IndexError on: 03-MAR-17 00:00:00, start: 5, end: 6
Got IndexError on: 03-MAR-17 00:00:00, start: 6, end: 7
Got IndexError on: 03-MAR-17 00:00:00, start: 7, end: 8
Got IndexError on: 03-MAR-17 00:00:00, start: 8, end: 9
Got IndexError on: 03-MAR-17 00:00:00, start: 9, end: 10
Got IndexError on: 03-MAR-17 00:00:00, start: 10, end: 11
Got IndexError on: 03-MAR-17 00:00:00, start: 11, end: 12
Got IndexErr

Got IndexError on: 06-MAR-17 00:00:00, start: 39, end: 40
Got IndexError on: 06-MAR-17 00:00:00, start: 40, end: 41
Got IndexError on: 06-MAR-17 00:00:00, start: 41, end: 42
Got IndexError on: 06-MAR-17 00:00:00, start: 42, end: 43
Got IndexError on: 06-MAR-17 00:00:00, start: 43, end: 44
Got IndexError on: 06-MAR-17 00:00:00, start: 44, end: 45
Got IndexError on: 06-MAR-17 00:00:00, start: 45, end: 46
Got IndexError on: 06-MAR-17 00:00:00, start: 46, end: 47
Got IndexError on: 06-MAR-17 00:00:00, start: 47, end: 48
Got IndexError on: 06-MAR-17 00:00:00, start: 48, end: 49
Got IndexError on: 06-MAR-17 00:00:00, start: 49, end: 50
Got IndexError on: 06-MAR-17 00:00:00, start: 50, end: 51
Got IndexError on: 06-MAR-17 00:00:00, start: 51, end: 52
Got IndexError on: 06-MAR-17 00:00:00, start: 52, end: 53
Got IndexError on: 06-MAR-17 00:00:00, start: 53, end: 54
Got IndexError on: 06-MAR-17 00:00:00, start: 54, end: 55
Got IndexError on: 06-MAR-17 00:00:00, start: 55, end: 56
Got IndexError

...

Got IndexError on: 28-FEB-17 00:00:00, start: 71, end: 72
Got IndexError on: 28-FEB-17 00:00:00, start: 72, end: 73
13-MAR-17 00:00:00
Got IndexError on: 13-MAR-17 00:00:00, start: 71, end: 72
Got IndexError on: 13-MAR-17 00:00:00, start: 72, end: 73
03-MAR-17 00:00:00
Got IndexError on: 03-MAR-17 00:00:00, start: 1, end: 2
Got IndexError on: 03-MAR-17 00:00:00, start: 2, end: 3
Got IndexError on: 03-MAR-17 00:00:00, start: 3, end: 4
Got IndexError on: 03-MAR-17 00:00:00, start: 4, end: 5
Got IndexError on: 03-MAR-17 00:00:00, start: 5, end: 6
Got IndexError on: 03-MAR-17 00:00:00, start: 6, end: 7
Got IndexError on: 03-MAR-17 00:00:00, start: 7, end: 8
Got IndexError on: 03-MAR-17 00:00:00, start: 8, end: 9
Got IndexError on: 03-MAR-17 00:00:00, start: 9, end: 10
Got IndexError on: 03-MAR-17 00:00:00, start: 10, end: 11
Got IndexError on: 03-MAR-17 00:00:00, start: 11, end: 12
Got IndexError on: 03-MAR-17 00:00:00, start: 12, end: 13
Got IndexError on: 03-MAR-17 00:00:00, start: 13, e

Got IndexError on: 01-MAR-17 00:00:00, start: 71, end: 72
Got IndexError on: 01-MAR-17 00:00:00, start: 72, end: 73
23-FEB-17 00:00:00
Got IndexError on: 23-FEB-17 00:00:00, start: 48, end: 49
Got IndexError on: 23-FEB-17 00:00:00, start: 49, end: 50
Got IndexError on: 23-FEB-17 00:00:00, start: 50, end: 51
Got IndexError on: 23-FEB-17 00:00:00, start: 51, end: 52
Got IndexError on: 23-FEB-17 00:00:00, start: 52, end: 53
Got IndexError on: 23-FEB-17 00:00:00, start: 53, end: 54
Got IndexError on: 23-FEB-17 00:00:00, start: 54, end: 55
Got IndexError on: 23-FEB-17 00:00:00, start: 55, end: 56
Got IndexError on: 23-FEB-17 00:00:00, start: 56, end: 57
Got IndexError on: 23-FEB-17 00:00:00, start: 57, end: 58
Got IndexError on: 23-FEB-17 00:00:00, start: 58, end: 59
Got IndexError on: 23-FEB-17 00:00:00, start: 59, end: 60
Got IndexError on: 23-FEB-17 00:00:00, start: 60, end: 61
Got IndexError on: 23-FEB-17 00:00:00, start: 61, end: 62
Got IndexError on: 23-FEB-17 00:00:00, start: 62, end

Got IndexError on: 12-APR-17 00:00:00, start: 71, end: 72
Got IndexError on: 12-APR-17 00:00:00, start: 72, end: 73
13-APR-17 00:00:00
Got IndexError on: 13-APR-17 00:00:00, start: 1, end: 2
Got IndexError on: 13-APR-17 00:00:00, start: 2, end: 3
Got IndexError on: 13-APR-17 00:00:00, start: 3, end: 4
Got IndexError on: 13-APR-17 00:00:00, start: 4, end: 5
Got IndexError on: 13-APR-17 00:00:00, start: 5, end: 6
Got IndexError on: 13-APR-17 00:00:00, start: 6, end: 7
Got IndexError on: 13-APR-17 00:00:00, start: 7, end: 8
Got IndexError on: 13-APR-17 00:00:00, start: 8, end: 9
Got IndexError on: 13-APR-17 00:00:00, start: 9, end: 10
Got IndexError on: 13-APR-17 00:00:00, start: 10, end: 11
Got IndexError on: 13-APR-17 00:00:00, start: 11, end: 12
Got IndexError on: 13-APR-17 00:00:00, start: 12, end: 13
Got IndexError on: 13-APR-17 00:00:00, start: 13, end: 14
Got IndexError on: 13-APR-17 00:00:00, start: 14, end: 15
Got IndexError on: 13-APR-17 00:00:00, start: 15, end: 16
Got IndexErr

Got IndexError on: 28-FEB-17 00:00:00, start: 17, end: 18
Got IndexError on: 28-FEB-17 00:00:00, start: 18, end: 19
Got IndexError on: 28-FEB-17 00:00:00, start: 19, end: 20
Got IndexError on: 28-FEB-17 00:00:00, start: 20, end: 21
Got IndexError on: 28-FEB-17 00:00:00, start: 26, end: 27
Got IndexError on: 28-FEB-17 00:00:00, start: 27, end: 28
Got IndexError on: 28-FEB-17 00:00:00, start: 30, end: 31
Got IndexError on: 28-FEB-17 00:00:00, start: 31, end: 32
Got IndexError on: 28-FEB-17 00:00:00, start: 32, end: 33
Got IndexError on: 28-FEB-17 00:00:00, start: 33, end: 34
Got IndexError on: 28-FEB-17 00:00:00, start: 34, end: 35
Got IndexError on: 28-FEB-17 00:00:00, start: 35, end: 36
Got IndexError on: 28-FEB-17 00:00:00, start: 36, end: 37
Got IndexError on: 28-FEB-17 00:00:00, start: 38, end: 39
Got IndexError on: 28-FEB-17 00:00:00, start: 39, end: 40
Got IndexError on: 28-FEB-17 00:00:00, start: 40, end: 41
Got IndexError on: 28-FEB-17 00:00:00, start: 41, end: 42
Got IndexError

Got IndexError on: 15-MAR-17 00:00:00, start: 46, end: 47
Got IndexError on: 15-MAR-17 00:00:00, start: 47, end: 48
Got IndexError on: 15-MAR-17 00:00:00, start: 48, end: 49
Got IndexError on: 15-MAR-17 00:00:00, start: 49, end: 50
Got IndexError on: 15-MAR-17 00:00:00, start: 50, end: 51
Got IndexError on: 15-MAR-17 00:00:00, start: 51, end: 52
Got IndexError on: 15-MAR-17 00:00:00, start: 52, end: 53
Got IndexError on: 15-MAR-17 00:00:00, start: 53, end: 54
Got IndexError on: 15-MAR-17 00:00:00, start: 54, end: 55
Got IndexError on: 15-MAR-17 00:00:00, start: 55, end: 56
Got IndexError on: 15-MAR-17 00:00:00, start: 56, end: 57
Got IndexError on: 15-MAR-17 00:00:00, start: 57, end: 58
Got IndexError on: 15-MAR-17 00:00:00, start: 58, end: 59
Got IndexError on: 15-MAR-17 00:00:00, start: 59, end: 60
Got IndexError on: 15-MAR-17 00:00:00, start: 60, end: 61
Got IndexError on: 15-MAR-17 00:00:00, start: 61, end: 62
Got IndexError on: 15-MAR-17 00:00:00, start: 62, end: 63
Got IndexError

In [14]:
new_df = new_df[new_df.toNextDistance != 'None']

In [15]:
new_df[['dayInterval', 'plannedPreTripDuration', 'plannedTripDuration', 'preTripDuration', 'toNextDistance', 'tripDuration', 'tripid']]

,dayInterval,plannedPreTripDuration,plannedTripDuration,preTripDuration,toNextDistance,tripDuration,tripid
0,16.0,0.0,42.0,0.0,400.0,34.0,4388888
1,16.0,42.0,70.0,34.0,600.0,60.0,4388888
2,16.0,112.0,19.0,94.0,200.0,18.0,4388888
3,16.0,131.0,101.0,126.0,800.0,89.0,4388888
4,16.0,232.0,50.0,215.0,400.0,52.0,4388888
5,16.0,282.0,51.0,277.0,400.0,56.0,4388888
6,16.0,333.0,30.0,363.0,200.0,32.0,4388888
7,16.0,363.0,56.0,407.0,400.0,52.0,4388888
8,16.0,419.0,72.0,468.0,600.0,46.0,4388888
9,16.0,491.0,78.0,543.0,600.0,59.0,4388888


In [4]:
#We can just only save it to file once!
new_df.to_csv("LR_2017_severalTrips_train_39A_b.csv")

NameError: name 'new_df' is not defined

In [5]:
new_df = pd.read_csv("LR_2017_severalTrips_train_39A_b.csv")

NameError: name 'pd' is not defined

In [3]:
new_df.dtypes

NameError: name 'new_df' is not defined

# Linear regression with package statsmodels

In [19]:
# BP neural networking training
import numpy as np
#Import statsmodels package for training a linear regression model.
import statsmodels.formula.api as sm


# X Descriptive features (6 continous features)
# y Target feature (happiness_class)

X = pd.concat([new_df[['preTripDuration',
                       'plannedPreTripDuration',
                       'plannedTripDuration',
                       'toNextDistance']]], axis=1)

y = new_df['tripDuration']
X = X.fillna(X.mean())
y = y.fillna(y.mean())

In [20]:
X.head()

,preTripDuration,plannedPreTripDuration,plannedTripDuration,toNextDistance
0,0.0,0.0,42.0,400.0
1,34.0,42.0,70.0,600.0
2,94.0,112.0,19.0,200.0
3,126.0,131.0,101.0,800.0
4,215.0,232.0,50.0,400.0


    Train Test Split
Let’s split our data into training and testing sets, this is done easily with SciKit Learn’s train_test_split function from model_selection:

In [21]:
# The formula specifies the target feature and the descriptive features used for training the 
# linear regression model.
# We first train a simple linear regression with only one descriptive feature (Size).
# For training the model we call the method fit() on the given data stored in our df dataframe.
lm = sm.ols(formula="tripDuration ~  preTripDuration + plannedPreTripDuration + plannedTripDuration + toNextDistance", data=new_df).fit()

# Print the model weights/parameters
print(lm.params)

Intercept                -4.346725
preTripDuration          -0.000574
plannedPreTripDuration    0.002673
plannedTripDuration       1.091435
toNextDistance           -0.022773
dtype: float64


In [22]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [23]:
X_train.count()

preTripDuration           4664
plannedPreTripDuration    4664
plannedTripDuration       4664
toNextDistance            4664
dtype: int64

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
# Instantiate model
lm2 = LinearRegression()

# Fit Model
lm2.fit(X_train, y_train)

# Predict
y_pred = lm2.predict(X_test)

# RMSE
print(np.sqrt(mean_squared_error(y_test, y_pred)))

50.5721224368


In [25]:
# Assume that:
# define index
i = 3808
col2 = new_df['tripDuration'].loc[i]
col3 = new_df['preTripDuration'].loc[i]
col4 = new_df['toNextDistance'].loc[i]
col5 = new_df['plannedPreTripDuration'].loc[i]
col6 = new_df['plannedTripDuration'].loc[i]
'''
X_new = pd.DataFrame({'preTripDuration':[col3],
                      'plannedPreTripDuration':[col5],
                      'plannedTripDuration':[col6],                    
                      'toNextDistance':[col4]})
'''
X_new = [col3, col5, col6, col4]
print("hasTakenTime:{}, toNextDistance:{}, plannedPreTripTime:{}, plannedTakenTime: {}, takeTime: {}".format(col3, col4, col5, col6, col2))
#print(X_new)

hasTakenTime:1698.0, toNextDistance:4300.0, plannedPreTripTime:1168.0, plannedTakenTime: 428.0, takeTime: 366.0


In [26]:
X_new

[1698.0, 1168.0, 428.0, 4300.0]

In [27]:
y_new = lm2.predict(np.array(X_new).reshape((1, -1)))
print(y_new[0])

367.218231937


    Data Preprocessing

The neural network in Python may have difficulty converging before the maximum number of iterations allowed if the data is not normalized. Multi-layer Perceptron is sensitive to feature scaling, so it is highly recommended to scale your data. Note that you must apply the same scaling to the test set for meaningful results. There are a lot of different methods for normalization of data, we will use the built-in StandardScaler for standardization

Refrer to:
https://www.springboard.com/blog/beginners-guide-neural-network-in-python-scikit-learn-0-18/

    Predictions and Evaluation

Now that we have a model it is time to use it to get predictions! We can do this simply with the predict() method off of our fitted model:

In [28]:
lm2.score(X_test, y_test)

0.56736375608910716

In [29]:
X_test.loc[1817]

preTripDuration           379.0
plannedPreTripDuration    402.0
plannedTripDuration        70.0
toNextDistance            600.0
Name: 1817, dtype: float64

In [30]:
np.array(X_test.loc[1817]).reshape((1, -1))

array([[ 379.,  402.,   70.,  600.]])

In [31]:
lm2.predict(np.array(X_test.loc[1817]).reshape((1, -1)))

array([ 59.87443889])

In [32]:
predict_df_test = pd.DataFrame({'ActualTripDuration': y_test, 'PredictedTripDuration': lm2.predict(X_test)})
predict_df_test

,ActualTripDuration,PredictedTripDuration
1817,43.0,59.874439
3808,366.0,367.218232
1392,26.0,47.501657
3071,16.0,50.976851
4933,28.0,56.879492
5988,127.0,93.505537
3884,35.0,37.517996
4972,16.0,24.835385
1676,71.0,84.387652
5444,23.0,49.949326


In [33]:
X_test.loc[1817]

preTripDuration           379.0
plannedPreTripDuration    402.0
plannedTripDuration        70.0
toNextDistance            600.0
Name: 1817, dtype: float64

In [34]:
new_df.loc[1817]

Unnamed: 0                   1875.0
dayInterval                    10.0
end_progrnumber                10.0
nextstoppointid              6112.0
plannedPreTripDuration        402.0
plannedTripDuration            70.0
preTripDuration               379.0
start_progrnumber               9.0
stoppointid                  6111.0
toNextDistance                600.0
tripDuration                   43.0
tripid                    4386521.0
Name: 1817, dtype: float64

In [50]:
predictions = lm2.predict(X_test)

In [51]:
print("Index | prediction |Actual result")

#for prediction in predictions:
 #   print(prediction)
count = 0
for i in y_test.index:
    if i == 247:
        print(X_test.loc[i], " VS ", y_test.loc[i])
    print(i, "\t", round(predictions[count],2),"\t", y_test.loc[i]) 
    count += 1

Index | prediction |Actual result
1817 	 59.87 	 43.0
3808 	 367.22 	 366.0
1392 	 47.5 	 26.0
3071 	 50.98 	 16.0
4933 	 56.88 	 28.0
5988 	 93.51 	 127.0
3884 	 37.52 	 35.0
4972 	 24.84 	 16.0
1676 	 84.39 	 71.0
5444 	 49.95 	 23.0
4621 	 57.34 	 25.0
729 	 40.68 	 47.0
5738 	 54.75 	 26.0
4364 	 31.72 	 23.0
2068 	 69.84 	 91.0
552 	 88.48 	 104.0
2489 	 42.8 	 39.0
3296 	 47.1 	 35.0
200 	 24.96 	 29.0
5274 	 22.79 	 11.0
1475 	 39.96 	 51.0
255 	 40.76 	 42.0
116 	 33.42 	 26.0
2000 	 69.45 	 83.0
3417 	 43.29 	 25.0
2523 	 71.96 	 169.0
6078 	 65.69 	 31.0
368 	 43.45 	 52.0
1242 	 53.57 	 41.0
2036 	 11.22 	 22.0
4846 	 51.49 	 51.0
5705 	 40.48 	 42.0
662 	 22.79 	 15.0
4977 	 13.48 	 9.0
3372 	 20.83 	 21.0
554 	 108.95 	 47.0
1206 	 42.51 	 42.0
1073 	 38.44 	 20.0
3601 	 32.14 	 28.0
5236 	 68.17 	 41.0
1131 	 54.04 	 100.0
1326 	 54.4 	 65.0
4913 	 26.26 	 18.0
3999 	 50.07 	 25.0
4864 	 452.05 	 277.0
5310 	 27.69 	 22.0
2879 	 64.69 	 56.0
1139 	 22.35 	 19.0
6207 	 95.

1669 	 25.99 	 20.0
4608 	 46.13 	 39.0
2839 	 23.13 	 18.0
5495 	 32.89 	 34.0
2406 	 46.73 	 67.0
3818 	 38.98 	 60.0
4831 	 63.29 	 35.0
3132 	 116.45 	 77.0
4205 	 49.83 	 112.0
1847 	 29.56 	 21.0
2349 	 30.78 	 20.0
2634 	 66.69 	 65.0
447 	 35.96 	 32.0
1989 	 97.96 	 81.0
6094 	 55.61 	 108.0
4497 	 39.0 	 48.0
2617 	 63.72 	 93.0
325 	 41.51 	 35.0
2399 	 26.32 	 21.0
685 	 83.22 	 39.0
605 	 25.78 	 42.0
5635 	 46.33 	 45.0
2209 	 63.79 	 122.0
5394 	 108.78 	 35.0
4741 	 341.71 	 391.0
714 	 54.43 	 40.0
1034 	 54.92 	 20.0
3871 	 39.24 	 29.0
1637 	 39.15 	 51.0
359 	 32.54 	 22.0
3014 	 36.13 	 34.0
2733 	 59.25 	 74.0
486 	 110.13 	 93.0
1198 	 39.29 	 30.0
5929 	 144.25 	 107.0
4423 	 24.02 	 31.0
3829 	 86.13 	 86.0
574 	 49.25 	 41.0
2478 	 68.77 	 40.0
5481 	 62.06 	 48.0
1926 	 67.44 	 19.0
3695 	 46.72 	 102.0
1117 	 53.63 	 55.0
4068 	 35.66 	 25.0
4996 	 61.24 	 69.0
3503 	 52.53 	 33.0
5301 	 96.94 	 106.0
968 	 65.86 	 88.0
5878 	 25.36 	 30.0
4314 	 26.1 	 14.0

1236 	 52.03 	 33.0
5584 	 68.14 	 79.0
2385 	 77.71 	 143.0
726 	 390.24 	 472.0
2032 	 39.33 	 35.0
4777 	 26.32 	 16.0
1850 	 99.08 	 80.0
5050 	 51.42 	 61.0
2666 	 57.5 	 32.0
5796 	 69.19 	 33.0
5773 	 40.42 	 70.0
275 	 50.21 	 86.0
2089 	 68.93 	 45.0
5109 	 61.54 	 86.0
2295 	 22.72 	 15.0
2784 	 60.98 	 37.0
3722 	 46.92 	 56.0
3265 	 44.33 	 44.0
6081 	 70.46 	 20.0
1102 	 58.85 	 30.0
329 	 24.27 	 16.0
1501 	 41.1 	 46.0
874 	 113.34 	 59.0
3671 	 56.74 	 104.0
3450 	 46.98 	 37.0
2652 	 127.28 	 131.0
3896 	 126.96 	 41.0
4908 	 61.35 	 64.0
4152 	 84.4 	 73.0
6008 	 53.02 	 44.0
996 	 12.64 	 12.0
5237 	 64.16 	 26.0
5117 	 26.28 	 22.0
796 	 56.0 	 69.0
123 	 22.62 	 18.0
2345 	 63.84 	 108.0
1513 	 43.78 	 42.0
3600 	 49.61 	 34.0
4995 	 47.69 	 25.0
4528 	 47.24 	 26.0
1331 	 25.03 	 20.0
3293 	 55.75 	 21.0
4181 	 40.13 	 27.0
5696 	 42.79 	 35.0
1882 	 41.16 	 51.0
1827 	 30.65 	 24.0
2883 	 47.99 	 54.0
5937 	 126.22 	 62.0
5873 	 84.08 	 73.0
1051 	 33.02 	 33.0
4

In [37]:
#from sklearn.metrics import classification_report,confusion_matrix
#print(confusion_matrix(y_test,predictions))

In [ ]:
#print(classification_report(y_test,predictions))

In [ ]:
'''
import pickle
file = "39A_2.clf"
output = open(file, 'wb')
pickle.dump(clf, output)
output.close()
'''

In [ ]:
file = "39A_2.clf"
pkl_file = open(file, 'rb')
new_clf = pickle.load(pkl_file)

In [141]:
print("Training set score: %f" % new_clf.score(X_test, y_test))

NameError: name 'new_clf' is not defined

In [150]:
def generalCrossValidationData(n_folds=10):
    '''
        This function is to separate DataFrame into k folds groups, each group include 90% train data and 10% test data,
        which is followed k-folds Cross-validation model.
        Input Value:
                - the number of folds, default is 10 folds
        Retrun Value:
                - ({k groups of train dataset}, {k group of test dataset})
    '''
    # Reading from a csv file, into a data frame
    # Drop 'customer' column because it is not continuous feature
    # Randomly shuffle the dataset
    # here you get 70% of the rows for train
    # this code tries to find that train = 70% and test = 30%
    df = pd.read_csv("LR_2017_severalTrips_train_39A_b.csv")
    #df = df.drop(['customer'], axis=1)
    new_df = pd.concat([df[['preTripDuration',
                       'plannedPreTripDuration',
                       'plannedTripDuration',
                       'toNextDistance',
                        'tripDuration']]], axis=1)


    #X = X.fillna(X.mean())
    #y = y.fillna(y.mean())

    # we use k-folds cross validation model and we separate the data into 10 groups of train & test datasets
    # we use KFold method to separate the dataset based 10-folds Cross-validation rule
    from sklearn.cross_validation import KFold
    kf = KFold(len(new_df), n_folds=n_folds)
    train_index = {}
    test_index = {}
    count = 0
    for indexA, indexB in kf:
        train_index[count] = indexA
        test_index[count] = indexB
        count += 1

    trains = {}
    tests = {}
    for i in range(10):
        trains[i] = new_df.loc[train_index[i]]
        tests[i] = new_df.loc[test_index[i]]
    return (trains, tests)

In [151]:
trains, tests = generalCrossValidationData(10)

In [152]:
scores = 0
for i in range(10):
    train = trains[i]
    test = tests[i]
    train_X = pd.concat([train[['preTripDuration',
                       'plannedPreTripDuration',
                       'plannedTripDuration',
                       'toNextDistance']]], axis=1)

    train_y = train['tripDuration']
    train_X = train_X.fillna(train_X.mean())
    train_y = train_y.fillna(train_y.mean())
    
    test_X = pd.concat([test[['preTripDuration',
                       'plannedPreTripDuration',
                       'plannedTripDuration',
                       'toNextDistance']]], axis=1)

    test_y = test['tripDuration']
    test_X = test_X.fillna(test_X.mean())
    test_y = test_y.fillna(test_y.mean())
    
    # Instantiate model
    lm2 = LinearRegression()

    # Fit Model
    lm2.fit(train_X, train_y)
    score = lm2.score(test_X, test_y)
    scores += score
    print("test {} Score: {}".format(i+1,score))
print("Average of Score is: ", scores/10)   

test 1 Score: 0.6682643767725698
test 2 Score: 0.550329450029216
test 3 Score: 0.3464347654298886
test 4 Score: 0.6183260583707555
test 5 Score: 0.5345635385473979
test 6 Score: 0.5292886616211663
test 7 Score: 0.5184196262977971
test 8 Score: 0.02100871472257626
test 9 Score: 0.6432681931158621
test 10 Score: 0.5759711071818743
Average of Score is:  0.500587449209
